讀取資料

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/store-sales-time-series-forecasting/train.csv")
holiday_event_df = pd.read_csv("/content/drive/MyDrive/store-sales-time-series-forecasting/holidays_events.csv")
stores_df = pd.read_csv("/content/drive/MyDrive/store-sales-time-series-forecasting/stores.csv")
transaction_df = pd.read_csv("/content/drive/MyDrive/store-sales-time-series-forecasting/transactions.csv")
oil_df = pd.read_csv("/content/drive/MyDrive/store-sales-time-series-forecasting/oil.csv")

擷取2015/08/15~2017/08/15的資料（共兩年）

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])
holiday_event_df['date'] = pd.to_datetime(holiday_event_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
transaction_df['date'] = pd.to_datetime(transaction_df['date'])
train_df = train_df[(train_df['date'] >= '2015-08-15') & (train_df['date'] <= '2017-12-31')]
holiday_event_df = holiday_event_df[(holiday_event_df['date'] >= '2015-08-15') & (holiday_event_df['date'] <= '2017-12-31')]
oil_df = oil_df[(oil_df['date'] >= '2015-08-15') & (oil_df['date'] <= '2017-12-31')]
transaction_df = transaction_df[(transaction_df['date'] >= '2015-08-15') & (transaction_df['date'] <= '2017-12-31')]

合併資料

In [ ]:
train_df = pd.merge(train_df, oil_df, on='date', how='left')
train_df = pd.merge(train_df, stores_df, on='store_nbr', how='left')
train_df = pd.merge(train_df, transaction_df, on=['date', 'store_nbr'], how='left')

In [ ]:
# holiday_event_df如果是National的資料就按照date合併；是其他的資料就按照date、city合併
national_df = holiday_event_df[holiday_event_df['locale'] == 'National']
train_national_df = pd.merge(train_df, national_df, on='date', how='left')
non_national_df = holiday_event_df[holiday_event_df['locale'] != 'National']
train_non_national_df = pd.merge(train_df, non_national_df, left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')
train_df = train_national_df.combine_first(train_non_national_df)

欄位名稱轉換（因為原始資料集有兩個type欄位分別在holiday_event_df跟stores_df）

---



In [ ]:
train_df = train_df.rename(columns={'type_x': 'store_type', 'type_y': 'event_type'})

特徵工程（1. 日期欄位拆分、2. 數值欄位 min-max normalization）

In [ ]:
# 日期欄位拆分
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['day_of_week'] = train_df['date'].dt.dayofweek

In [ ]:
# 數值欄位 min-max normalization
columns_to_normalize = ['onpromotion', 'dcoilwtico', 'transactions']
train_df[columns_to_normalize] = (train_df[columns_to_normalize] - train_df[columns_to_normalize].min()) / (train_df[columns_to_normalize].max() - train_df[columns_to_normalize].min())

In [ ]:
train_df

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,store_type,...,transactions,event_type,locale,locale_name,description,transferred,year,month,day,day_of_week
0,1700028,2015-08-15,1,AUTOMOTIVE,3.000,0.000000,NaN,Quito,Pichincha,D,...,0.164472,NaN,NaN,NaN,NaN,NaN,2015,8,15,5
1,1700029,2015-08-15,1,BABY CARE,0.000,0.000000,NaN,Quito,Pichincha,D,...,0.164472,NaN,NaN,NaN,NaN,NaN,2015,8,15,5
2,1700030,2015-08-15,1,BEAUTY,0.000,0.000000,NaN,Quito,Pichincha,D,...,0.164472,NaN,NaN,NaN,NaN,NaN,2015,8,15,5
3,1700031,2015-08-15,1,BEVERAGES,2845.000,0.006748,NaN,Quito,Pichincha,D,...,0.164472,NaN,NaN,NaN,NaN,NaN,2015,8,15,5
4,1700032,2015-08-15,1,BOOKS,0.000,0.000000,NaN,Quito,Pichincha,D,...,0.164472,NaN,NaN,NaN,NaN,NaN,2015,8,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306201,3000883,2017-08-15,9,POULTRY,438.133,0.000000,0.755744,Quito,Pichincha,B,...,0.257362,NaN,NaN,NaN,NaN,NaN,2017,8,15,1
1306202,3000884,2017-08-15,9,PREPARED FOODS,154.553,0.001350,0.755744,Quito,Pichincha,B,...,0.257362,NaN,NaN,NaN,NaN,NaN,2017,8,15,1
1306203,3000885,2017-08-15,9,PRODUCE,2419.729,0.199730,0.755744,Quito,Pichincha,B,...,0.257362,NaN,NaN,NaN,NaN,NaN,2017,8,15,1
1306204,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,0.010796,0.755744,Quito,Pichincha,B,...,0.257362,NaN,NaN,NaN,NaN,NaN,2017,8,15,1


train_df['family']、train_df['description']欄位用bert模型轉成詞向量

In [ ]:
# family欄位轉為embedding
from transformers import BertModel, BertTokenizer
import torch
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
unique_families = train_df['family'].unique()
inputs = tokenizer(list(unique_families), padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

family_embedding_map = {family: embedding.tolist() for family, embedding in zip(unique_families, embeddings)}
train_df['bert_embeddings'] = train_df['family'].map(family_embedding_map)

In [ ]:
train_df = train_df.rename(columns={'bert_embeddings': 'family_bert_embeddings'})

In [ ]:
# description欄位轉為embedding
from transformers import BertModel, BertTokenizer
import torch
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
train_df['description'] = train_df['description'].astype(str)

unique_description = train_df['description'].dropna().unique()
inputs = tokenizer(list(unique_description), padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

description_embedding_map = {description: embedding.tolist() for description, embedding in zip(unique_description, embeddings)}
train_df['description_bert_embeddings'] = train_df['description'].map(description_embedding_map)

新增經緯度欄位

In [ ]:
cities_coordinates = {
    "Quito": {"lat": -0.1806532, "lon": -78.4678382},
    "Guayaquil": {"lat": -2.1709979, "lon": -79.9223592},
    "Santo Domingo": {"lat": -0.2530494, "lon": -79.1753765},
    "Cuenca": {"lat": -2.9001285, "lon": -79.0058965},
    "Latacunga": {"lat": -0.9393387, "lon": -78.6155545},
    "Manta": {"lat": -0.9676533, "lon": -80.7089101},
    "Machala": {"lat": -3.2581112, "lon": -79.9553924},
    "Ambato": {"lat": -1.2416666, "lon": -78.6195459},
    "Quevedo": {"lat": -1.0225124, "lon": -79.4604035},
    "Esmeraldas": {"lat": 0.9681789, "lon": -79.6517202},
    "Loja": {"lat": -3.9931283, "lon": -79.2042216},
    "Libertad": {"lat": -2.2311612, "lon": -80.9008852},
    "Playas": {"lat": -2.6284683, "lon": -80.3895886},
    "Daule": {"lat": -1.8621807, "lon": -79.9776688},
    "Babahoyo": {"lat": -1.8019264, "lon": -79.5346458},
    "Cayambe": {"lat": 0.0430556, "lon": -78.1459943},
    "Salinas": {"lat": -2.2171001, "lon": -80.9586051},
    "Puyo": {"lat": -1.4923925, "lon": -78.0024134},
    "Guaranda": {"lat": -1.5904732, "lon": -79.0022925},
    "Ibarra": {"lat": 0.3517083, "lon": -78.1223373},
    "Riobamba": {"lat": -1.6635508, "lon": -78.654646},
    "El Carmen": {"lat": -0.2687816, "lon": -79.466199}
}

train_df['coordinates'] = train_df['city'].map(cities_coordinates)

train_df['longitude'] = train_df['coordinates'].map(lambda x: x['lon'] if x is not None else None)
train_df['latitude'] = train_df['coordinates'].map(lambda x: x['lat'] if x is not None else None)

train_df.drop('coordinates', axis=1, inplace=True)

把sales欄位放到資料集最後，作為y_train

In [ ]:
cols = [col for col in train_df.columns if col != 'sales']
cols.append('sales')
train_df = train_df[cols]

In [ ]:
train_df

,id,date,store_nbr,family,onpromotion,dcoilwtico,city,state,store_type,cluster,...,transferred,year,month,day,day_of_week,family_bert_embeddings,description_bert_embeddings,longitude,latitude,sales
0,1700028,2015-08-15,1,AUTOMOTIVE,0.000000,NaN,Quito,Pichincha,D,13,...,NaN,2015,8,15,5,"[-0.1256479024887085, 0.09700357913970947, -0....","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,3.000
1,1700029,2015-08-15,1,BABY CARE,0.000000,NaN,Quito,Pichincha,D,13,...,NaN,2015,8,15,5,"[-0.02602454461157322, -0.017557932063937187, ...","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,0.000
2,1700030,2015-08-15,1,BEAUTY,0.000000,NaN,Quito,Pichincha,D,13,...,NaN,2015,8,15,5,"[-0.27227234840393066, 0.08034408092498779, -0...","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,0.000
3,1700031,2015-08-15,1,BEVERAGES,0.006748,NaN,Quito,Pichincha,D,13,...,NaN,2015,8,15,5,"[-0.342194527387619, 0.4779341220855713, -0.22...","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,2845.000
4,1700032,2015-08-15,1,BOOKS,0.000000,NaN,Quito,Pichincha,D,13,...,NaN,2015,8,15,5,"[-0.1032329872250557, 0.15401753783226013, -0....","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306201,3000883,2017-08-15,9,POULTRY,0.000000,0.755744,Quito,Pichincha,B,6,...,NaN,2017,8,15,1,"[-0.5122681260108948, 0.09797710180282593, -0....","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,438.133
1306202,3000884,2017-08-15,9,PREPARED FOODS,0.001350,0.755744,Quito,Pichincha,B,6,...,NaN,2017,8,15,1,"[0.0060926396399736404, 0.27368366718292236, -...","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,154.553
1306203,3000885,2017-08-15,9,PRODUCE,0.199730,0.755744,Quito,Pichincha,B,6,...,NaN,2017,8,15,1,"[-0.5437078475952148, 0.41101962327957153, -0....","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,2419.729
1306204,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,0.010796,0.755744,Quito,Pichincha,B,6,...,NaN,2017,8,15,1,"[-0.2551138997077942, 0.32630041241645813, -0....","[-0.04250179976224899, 0.03686172887682915, 0....",-78.467838,-0.180653,121.000


In [ ]:
# import pandas as pd

# # Ensure the 'date' column is in datetime format
# train_df['date'] = pd.to_datetime(train_df['date'])

# # Filter the DataFrame for each year and save to CSV
# for year in [2015, 2016, 2017]:
#     df_year = train_df[train_df['date'].dt.year == year]
#     df_year.to_csv(f"DM Final Project training data {year}.csv.gz", compression='gzip', index=False)

下面不用用到：用來計算testing data的transactions欄位

In [ ]:
# train_df['date'] = pd.to_datetime(train_df['date'])
# august_days_df = train_df[(train_df['date'].dt.month == 8) & (train_df['date'].dt.day >= 16) & (train_df['date'].dt.day <= 31)]
# daily_avg_transactions = august_days_df.groupby([august_days_df['date'].dt.day, 'store_nbr'])['transactions'].mean()
# print(daily_avg_transactions)

date  store_nbr
16    1            0.125329
      2            0.218698
      3            0.388197
      4            0.170158
      5            0.152203
                     ...   
31    50           0.281901
      51           0.194817
      52           0.531243
      53           0.123175
      54           0.097079
Name: transactions, Length: 864, dtype: float64


In [ ]:
# df = daily_avg_transactions.reset_index()
# df.columns = ['date', 'store_nbr', 'transactions']
# df['date'] = df['date'].apply(lambda day: pd.Timestamp(f'2017-08-{day:02d}'))
# print(df)

          date  store_nbr  transactions
0   2017-08-16          1      0.125329
1   2017-08-16          2      0.218698
2   2017-08-16          3      0.388197
3   2017-08-16          4      0.170158
4   2017-08-16          5      0.152203
..         ...        ...           ...
859 2017-08-31         50      0.281901
860 2017-08-31         51      0.194817
861 2017-08-31         52      0.531243
862 2017-08-31         53      0.123175
863 2017-08-31         54      0.097079

[864 rows x 3 columns]


In [ ]:
df.to_csv("testing_transactions.csv", index=False)